# Hosting Strands Agents with OpenAI models in Amazon Bedrock AgentCore Runtime

## Overview

In this tutorial we will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime. 

We will focus on a Strands Agents with OpenAI model example. For Strands Agents with Amazon Bedrock model check [here](../01-strands-with-bedrock-model) and 
for LangGraph with Amazon Bedrock model check [here](../02-langgraph-with-bedrock-model)


### Tutorial Details

| Information         | Details                                                                  |
|:--------------------|:-------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                           |
| Agent type          | Single                                                                   |
| Agentic Framework   | Strands Agents                                                           |
| LLM model           | GPT 4.1 mini                                                             |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands Agent and OpenAI Model |
| Tutorial vertical   | Cross-vertical                                                           |
| Example complexity  | Easy                                                                     |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                             |

### Tutorial Architecture

In this tutorial we will describe how to deploy an existing agent to AgentCore runtime. 

For demonstration purposes, we will  use a Strands Agent using Amazon Bedrock models

In our example we will use a very simple agent with two tools: `get_weather` and `get_time`. 

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="60%"/>
</div>

### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using OpenAI models
* Using Strands Agents


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker running

In [ ]:
#!uv pip install --force-reinstall -U -r requirements.txt --quiet

## Creating your agents and experimenting locally

Before we deploy our agents to AgentCore Runtime, let's develop and run them locally for experimentation purposes.

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look how each agent is developed during the experimentation phase.

The architecture here will look as following:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="60%"/>
</div>

In [ ]:
%%writefile strands_agents_openai.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from strands.models.openai import OpenAIModel
import os
import boto3
from functools import cache

ssm = boto3.client('ssm')
#reference https://strandsagents.com/latest/documentation/docs/user-guide/concepts/model-providers/openai/
@cache
def get_ssm_parameter(parameter_name):
    """Retrieve secure string from AWS Parameter Store"""
    
    response = ssm.get_parameter(
        Name=parameter_name,
        WithDecryption=True
    )
    return response['Parameter']['Value']

# Get OpenAI API key from Parameter Store

TEMPERATURE = 1
# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

model = "gpt-5-mini"
openai_model = OpenAIModel(
    client_args = {
        "api_key": get_ssm_parameter('/openai/api_key')
    },
    model_id=model, 
    params={ "temperature": TEMPERATURE}
)


agent = Agent(
    model=openai_model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

def strands_agent_open_ai(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_open_ai(json.loads(args.payload))
    print(response)

#### Invoking local agent

In [ ]:
!python strands_agents_openai.py '{"prompt": "What is the weather now?"}'

## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to AgentCore Runtime. To do so we need to:
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`

### Strands Agents with OpenAI model
Let's start with our Strands Agent using the GPT 4.1 mini model. All the others will work exactly the same.

In [ ]:
%%writefile strands_agents_openai.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from strands.models.openai import OpenAIModel
import os
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from functools import cache

import os
import boto3

ssm = boto3.client('ssm')
#reference https://strandsagents.com/latest/documentation/docs/user-guide/concepts/model-providers/openai/

@cache
def get_ssm_parameter(parameter_name):
    """Retrieve secure string from AWS Parameter Store"""
    
    response = ssm.get_parameter(
        Name=parameter_name,
        WithDecryption=True
    )
    return response['Parameter']['Value']

# Get OpenAI API key from Parameter Store

TEMPERATURE = 1

app = BedrockAgentCoreApp()


# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

model = "gpt-5-mini"
openai_model = OpenAIModel(
    client_args = {
        "api_key": get_ssm_parameter('/openai/api_key')
    },
    model_id=model, 
    params={ "temperature": TEMPERATURE}
)


agent = Agent(
    model=openai_model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)


@app.entrypoint
def strands_agent_open_ai(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCode Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Configure AgentCore Runtime deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import boto3
import json
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()

agent_name = "strands_openai"
response = agentcore_runtime.configure(
    # Add parameter store permissions to execution role  
    entrypoint="strands_agents_openai.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.openai.txt",
    region=region,
    agent_name=agent_name
)
response

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

<div style="text-align:left">
    <img src="images/invoke.png" width=85%"/>
</div>

In [ ]:
status_response.config.execution_role

In [ ]:
import boto3
import json
role_name = status_response.config.execution_role.split("/")[-1]
# The role name created by the toolkit (replace with actual role name if different)
iam_client = boto3.client('iam')
# Attach the AmazonSSMReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/AmazonSSMReadOnlyAccess"
iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn
)

kms_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "kms:Decrypt"
            ],
            "Resource": f"arn:aws:kms:{region}:{boto3.client('sts').get_caller_identity()['Account']}:key/*",
            "Condition": {
                "StringEquals": {
                    "kms:CallerAccount": boto3.client('sts').get_caller_identity()['Account'],
                    "kms:KeyAlias": "alias/aws/ssm"
                }
            }
        }
    ]
}

# Add the inline KMS policy to the role
iam_client.put_role_policy(
    RoleName=role_name,
    PolicyName="KMSDecryptForSSM",
    PolicyDocument=json.dumps(kms_policy)
)


In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "Hi, what can you do?"})
invoke_response

### Processing invocation results

We can now process our invocation results to include it in an application

In [ ]:
from IPython.display import Markdown, display
import json
response_text = invoke_response['response'][0]
display(Markdown(response_text))

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

In [ ]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "How much is 2X2?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# Congratulations!